<a href="https://colab.research.google.com/github/Aman-Gautam1/Assignment_ZeoTap/blob/main/Aman_Gautam_LookALike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries  and code

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [139]:
cust_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Assignment/Assignment Zeotap/Customers.csv')
prod_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Assignment/Assignment Zeotap/Products.csv')
trans_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Assignment/Assignment Zeotap/Transactions.csv')

# Merging the dataset

In [140]:
merged_df = trans_df.merge(cust_df[['CustomerID', 'Region']], on='CustomerID', how='left') \
                    .merge(prod_df[['ProductID', 'Category', 'Price']], on='ProductID', how='left')
merged_df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,Region,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Europe,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Asia,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Europe,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,South America,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Europe,Electronics,300.68
...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,South America,Electronics,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,North America,Electronics,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,North America,Electronics,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Asia,Electronics,459.86


In [141]:
merged_df.drop('Price_y',axis=1,inplace=True)
merged_df.rename(columns={'Price_x':'Price'},inplace=True)
merged_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,Region,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Europe,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Asia,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Europe,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,South America,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Europe,Electronics


In [142]:
merged_df.isnull().sum()

,0
TransactionID,0
CustomerID,0
ProductID,0
TransactionDate,0
Quantity,0
TotalValue,0
Price,0
Region,0
Category,0


# Feature Engineering

In [144]:
#creating customer inputs
customer_profiles = merged_df.groupby('CustomerID').agg({
    'ProductID': pd.Series.nunique,   # Number of unique products bought
    'TotalValue': 'sum',               # Total amount spent by the customer
    'Quantity': 'sum',                 # Total quantity bought
}).reset_index()

# Add region information from the customer dataset
customer_profiles = pd.merge(customer_profiles, cust_df[['CustomerID', 'Region']], on='CustomerID', how='left')

 **Creating** **Purchasing**  **power** **bracket**

In [145]:
# Calculate total spending for each customer
customer_spending = merged_df.groupby('CustomerID')['TotalValue'].sum()

# Define purchasing power brackets (e.g., low, medium, high)
def categorize_spending(total_value):
    if total_value < 2500:
        return 'Low'
    elif total_value < 4500:
        return 'Medium'
    else:
        return 'High'

customer_profiles['PurchasingPower'] = customer_profiles['TotalValue'].apply(categorize_spending)

# View the updated customer profiles
customer_profiles.head()


,CustomerID,ProductID,TotalValue,Quantity,Region,PurchasingPower
0,C0001,5,3354.52,12,South America,Medium
1,C0002,4,1862.74,10,Asia,Low
2,C0003,4,2725.38,14,South America,Medium
3,C0004,8,5354.88,23,South America,High
4,C0005,3,2034.24,7,Asia,Low


In [146]:
customer_profiles.PurchasingPower.value_counts()

,count
PurchasingPower,
Medium,78
Low,63
High,58


# Encoding of Region

In [147]:
# One-hot encode categorical features like 'Region'
customer_profiles_encoded = pd.get_dummies(customer_profiles, columns=['Region', 'PurchasingPower'], drop_first=True)


customer_profiles_encoded.head()

,CustomerID,ProductID,TotalValue,Quantity,Region_Europe,Region_North America,Region_South America,PurchasingPower_Low,PurchasingPower_Medium
0,C0001,5,3354.52,12,False,False,True,False,True
1,C0002,4,1862.74,10,False,False,False,True,False
2,C0003,4,2725.38,14,False,False,True,False,True
3,C0004,8,5354.88,23,False,False,True,False,False
4,C0005,3,2034.24,7,False,False,False,True,False


#Standardization

In [148]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
customer_profiles_encoded[['ProductID', 'TotalValue', 'Quantity']] = scaler.fit_transform(customer_profiles_encoded[['ProductID', 'TotalValue', 'Quantity']])

# Display the standardized data
customer_profiles_encoded.head()

,CustomerID,ProductID,TotalValue,Quantity,Region_Europe,Region_North America,Region_South America,PurchasingPower_Low,PurchasingPower_Medium
0,C0001,0.050047,-0.061701,-0.122033,False,False,True,False,True
1,C0002,-0.424204,-0.877744,-0.448000,False,False,False,True,False
2,C0003,-0.424204,-0.405857,0.203934,False,False,True,False,True
3,C0004,1.472798,1.032547,1.670787,False,False,True,False,False
4,C0005,-0.898455,-0.783929,-0.936951,False,False,False,True,False


# Use of cosine Similarity

In [149]:
from sklearn.metrics.pairwise import cosine_similarity

# Select the relevant columns for similarity calculation
features = customer_profiles_encoded.drop(['CustomerID'], axis=1)  # Exclude CustomerID column
# Optionally include or exclude any categorical columns if necessary
# features = customer_profiles_encoded[['TotalValue', 'Quantity', 'Region_Europe', 'Region_North America', ...]]

# Compute the Cosine Similarity matrix
similarity_matrix = cosine_similarity(features)

# Convert it to a DataFrame for easier inspection
similarity_df = pd.DataFrame(similarity_matrix, index=customer_profiles_encoded['CustomerID'], columns=customer_profiles_encoded['CustomerID'])

# Display the similarity matrix
print(similarity_df.head())


CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.042011  0.901085  0.213899  0.045593  0.899597   
C0002       0.042011  1.000000  0.196337 -0.586339  0.934238 -0.057787   
C0003       0.901085  0.196337  1.000000  0.072505  0.181115  0.835619   
C0004       0.213899 -0.586339  0.072505  1.000000 -0.768015  0.147281   
C0005       0.045593  0.934238  0.181115 -0.768015  1.000000  0.060622   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001       0.467124  0.217293  0.061322  0.021024  ...  0.984978  0.429827   
C0002       0.484076 -0.430831  0.839118  0.802028  ...  0.154305  0.816828   
C0003       0.565676  0.007438  0.150335  0.199489  ...  0.894023  0.470247   
C0004      -0.723583  0.735568 -0.757667 -0.388615  ...  0.085528 -0.350969   
C0005  

# Recommend top3 lookalikes

In [150]:
lookalike_recommendations = {}

# Loop through the first 20 customers (C0001 - C0020)
for customer_id in customer_profiles_encoded['CustomerID'][:20]:
    # Get the similarity scores for this customer
    similarities = similarity_df[customer_id]

    # Sort the customers based on similarity (ignore the first one as it's the customer itself)
    sorted_similarities = similarities.sort_values(ascending=False)
    top_3_similar_customers = sorted_similarities.index[1:4]  # Get the top 3 similar customers

    # Create a list of tuples (customer_id, similarity_score)
    lookalike_recommendations[customer_id] = [
        (lookalike, sorted_similarities[lookalike]) for lookalike in top_3_similar_customers
    ]

# Print out the lookalike recommendations
for customer_id, recommendations in lookalike_recommendations.items():
    print(f"Customer {customer_id}:")
    for lookalike, score in recommendations:
        print(f"\tLookalike: {lookalike}, Similarity Score: {score}")


Customer C0001:
	Lookalike: C0107, Similarity Score: 0.994632640008671
	Lookalike: C0137, Similarity Score: 0.9936381382351789
	Lookalike: C0191, Similarity Score: 0.9849776082580975
Customer C0002:
	Lookalike: C0142, Similarity Score: 0.9885438560220585
	Lookalike: C0186, Similarity Score: 0.9473006501939832
	Lookalike: C0043, Similarity Score: 0.947149042386917
Customer C0003:
	Lookalike: C0133, Similarity Score: 0.9480793412710703
	Lookalike: C0190, Similarity Score: 0.9473431895624816
	Lookalike: C0174, Similarity Score: 0.9262718596095505
Customer C0004:
	Lookalike: C0113, Similarity Score: 0.9887735395871667
	Lookalike: C0104, Similarity Score: 0.9643394618678176
	Lookalike: C0102, Similarity Score: 0.9639365891122683
Customer C0005:
	Lookalike: C0186, Similarity Score: 0.9974522198413518
	Lookalike: C0159, Similarity Score: 0.9969275009402225
	Lookalike: C0140, Similarity Score: 0.9858906900154507
Customer C0006:
	Lookalike: C0048, Similarity Score: 0.9418930930117574
	Lookalike

In [151]:
lookalike_recommendations = []

# Loop through the first 20 customers (C0001 - C0020)
for customer_id in customer_profiles_encoded['CustomerID'][:20]:
    # Get the similarity scores for the current customer
    similarities = similarity_df[customer_id]

    # Sort the customers by similarity, excluding the customer itself (similarity 1)
    sorted_similarities = similarities.sort_values(ascending=False)
    top_3_similar_customers = sorted_similarities.index[1:4]  # Top 3 excluding the current customer

    # Create a list of tuples (lookalike, similarity score)
    similar_customers = [(lookalike, sorted_similarities[lookalike]) for lookalike in top_3_similar_customers]

    # Append the customer ID and their top 3 similar customers with similarity scores to the list
    lookalike_recommendations.append({
        'CustomerID': customer_id,
        'Lookalikes': str(similar_customers)  # Convert the list of tuples to string format
    })

# Create a DataFrame from the list of lookalike recommendations
lookalike_df = pd.DataFrame(lookalike_recommendations)

In [152]:
lookalike_df.head()

,CustomerID,Lookalikes
0,C0001,"[('C0107', 0.994632640008671), ('C0137', 0.993..."
1,C0002,"[('C0142', 0.9885438560220585), ('C0186', 0.94..."
2,C0003,"[('C0133', 0.9480793412710703), ('C0190', 0.94..."
3,C0004,"[('C0113', 0.9887735395871667), ('C0104', 0.96..."
4,C0005,"[('C0186', 0.9974522198413518), ('C0159', 0.99..."


In [153]:
lookalike_df.to_csv('Lookalike.csv', index=False)